# Notebook d'inference

Ce notebook qui contient un script d'inférence est à transformer en webapp pour effectuer des prédictions,

Vous y trouverez toutes les ressources nécessaires pour remplir le stub de `app.py` avec les bonnes valeurs

In [ ]:
# !pip install -r requirements.txt

In [ ]:
import time
from pathlib import Path
from typing import Dict
import torch

from PIL import Image

In [ ]:
MODEL_NAMES = ["yolov5s", "yolov5m", "yolov5l"]

In [ ]:
def load_model(model_name: str) -> Dict:
    # Load model from torch
    model = torch.hub.load("ultralytics/yolov5", model_name, pretrained=True)
    # Evaluation mode
    model = model.eval()

    return model

In [ ]:
model = load_model("yolov5s")

In [1]:
# img = Path("cats.jpg")
# img = Image.open(img)

import requests
from io import BytesIO

response = requests.get("https://icatcare.org/app/uploads/2018/07/Thinking-of-getting-a-cat.png")
img = Image.open(BytesIO(response.content))

if img.mode == "RGBA":
    img = img.convert("RGB")

NameError: name 'Image' is not defined

In [ ]:
# Inference
t0 = time.time()
predictions = model(img, size=640)  # includes NMS
t1 = time.time()
classes = predictions.names

preds = predictions.xyxy[0].numpy()

In [ ]:
preds

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import  ImageDraw, ImageFont


def draw_preds(image, preds, class_names):

    image = image.copy()

    colors = plt.cm.get_cmap("viridis", len(class_names)).colors
    colors = (colors[:, :3] * 255.0).astype(np.uint8)

    font = list(Path("/usr/share/fonts").glob("**/*.ttf"))[0].name
    font = ImageFont.truetype(font=font, size=np.floor(3e-2 * image.size[1] + 0.5).astype("int32"))
    thickness = (image.size[0] + image.size[1]) // 300

    for i in range(len(preds)):
        box = preds[i, :4]
        score = float(preds[i, 4])
        class_idx = int(preds[i, 5])
        predicted_class = class_names[class_idx]

        label = "{} {:.2f}".format(predicted_class, score)

        draw = ImageDraw.Draw(image)
        label_size = draw.textsize(label, font)

        left, top, right, bottom = box
        top = max(0, np.floor(top + 0.5).astype("int32"))
        left = max(0, np.floor(left + 0.5).astype("int32"))
        bottom = min(image.size[1], np.floor(bottom + 0.5).astype("int32"))
        right = min(image.size[0], np.floor(right + 0.5).astype("int32"))
        print(label, (left, top), (right, bottom))

        if top - label_size[1] >= 0:
            text_origin = np.array([left, top - label_size[1]])
        else:
            text_origin = np.array([left, top + 1])

        # My kingdom for a good redistributable image drawing library.
        for r in range(thickness):
            draw.rectangle([left + r, top + r, right - r, bottom - r], outline=tuple(colors[class_idx]))
        draw.rectangle([tuple(text_origin), tuple(text_origin + label_size)], fill=tuple(colors[class_idx]))
        draw.text(text_origin, label, fill=(0, 0, 0), font=font)
        del draw

    return image

In [ ]:
image_with_preds = draw_preds(img, preds, classes)

In [ ]:
from IPython.display import display  # to display images

display(image_with_preds)

In [ ]:
image_with_preds.save("test.png")